<a href="https://colab.research.google.com/github/indra622/tutorials/blob/master/Multi_AI_Agent_Study/Part2_Ch4_02_basic_implementation_with_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai python-dotenv

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [5]:
from openai import OpenAI

client = OpenAI()
client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {
          "role": "user",
          "content": "2002년 월드컵 4강 국가 알려줘"
        }
    ]
)

ChatCompletion(id='chatcmpl-AuqJF7YKQ1DMjnSgCP6Idq2rdDWTI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2002년 FIFA 월드컵 4강에 진출한 국가는 한국, 독일, 브라질, 그리스입니다. 한국과 독일, 브라질과 그리스가 준결승에 진출하여 경기를 치뤘습니다. 결국 한국은 4강에서 독일에 패배하고, 브라질은 그리스에 승리하여 결승에 진출했습니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738110809, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=134, prompt_tokens=31, total_tokens=165, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [6]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00


In [8]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
)

chat.invoke("안녕 너를 소개해줄래?")

AIMessage(content='안녕하세요! 제 이름은 AI 어시스턴트 입니다. 제가 여러분들의 질문에 대답해드리고 도와드릴 수 있어요. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 22, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a3d353d1-1251-4033-b2b6-64cfa37b5647-0', usage_metadata={'input_tokens': 22, 'output_tokens': 64, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template(
        """
        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고,
        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
    )
)

prompt

PromptTemplate(input_variables=['개수', '재료'], input_types={}, partial_variables={}, template='\n        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고,\n        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        {재료}\n        ')

In [10]:
prompt.invoke({"개수":6, "재료":"사과, 잼"})

StringPromptValue(text='\n        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 6추천하고,\n        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        사과, 잼\n        ')

In [11]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, name 부여
        ("system", " You are a helpful AI bot. Your name is {name}."),
        # 서로 안부를 묻고 답하는 히스토리를 미리 주입(few-shot)
        ("human", "Hello, how are you doing?"),
        ("ai", "i'm doing well, thanks!"),
        #실제 사용자 입력 전달
        ("human", "{user_input}"),

    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
messages

[SystemMessage(content=' You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="i'm doing well, thanks!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

In [12]:
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model | StrOutputParser()

chain.invoke({"topic": "bear"})

'Why do bears have hairy coats? \n\nBecause they look silly in sweaters!'

In [13]:
model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model

for s in chain.stream({"topic":"bears"}):
  print(s.content, end="", flush=True)

Why do bears have sticky paws? 

Because they always bear-hug the honey!

In [17]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                '너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘.'
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)
model = ChatOpenAI(model = 'gpt-4o-mini')
chain = chat_template | model
chain.invoke("액션")

AIMessage(content='["다크 나이트", "존 윅", "매드맥스: 분노의 도로"]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 72, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-7623416a-69c2-488b-aef4-e46fb092e482-0', usage_metadata={'input_tokens': 72, 'output_tokens': 24, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [19]:
prompt = PromptTemplate(
    template = "List {subject}. answer in Korean \n {format_instructions}",
    input_variables = ["subject"],
    partial_variables = {"format_instructions": format_instructions},
)

model = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | model | output_parser
chain.invoke({'subject': "공포 영화"})


['더 링',
 '미스트',
 '사탄의 인형',
 '괴물',
 '귀신이 산다',
 '인시디어스',
 '툼 레이더',
 '커뮤니티',
 '엑소시스트',
 '컨저링']

In [21]:
from typing import List

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Country(BaseModel):
  continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
  population: str = Field(description ="사용자가 물어본 나라의 인구(int 형식)")

parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query. \n {format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke("아르헨티나는 어떤 나라야?")

{'continent': 'South America', 'population': '45 million'}

In [23]:

from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")

'안녕하세요'

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("""다음 한글 문장을 프랑스어로 번역해줘 {sentence}
French sentence: (print from here)""")

model = ChatOpenAI(model="gpt-4o-mini")

runnable_chain = {"sentence": RunnablePassthrough()} | prompt | model | StrOutputParser()

runnable_chain.invoke("안녕하세요")

'안녕하세요는 프랑스어로 "Bonjour"입니다.'

In [26]:
(RunnablePassthrough.assign(mult=lambda x: x["num"]*3)).invoke({"num":3})

{'num': 3, 'mult': 9}

In [27]:
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x['num']*3),
    modified=lambda x: x['num'] + 1,
)

runnable.invoke({"num":1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [28]:
def add_smile(x):
  return x+ ":)"

In [29]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(add_smile)


In [30]:
prompt_str = "{topic}의 역사에 대해 세문장으로 설명해 주세요"
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | StrOutputParser()

In [32]:
chain = prompt | model | StrOutputParser() | runnable

chain.invoke("반도체")

'반도체의 역사는 19세기 말에 반도체 소재가 발견되면서 시작되었으며, 초기에는 주로 연구용으로 사용되었다. 20세기 중반, 트랜지스터의 발명으로 반도체 기술이 급격히 발전하며 컴퓨터 및 전자기기의 핵심 부품으로 자리잡았다. 이후 집적회로(IC)와 마이크로프로세서의 개발로 반도체 산업은 폭발적인 성장을 이루어 현재의 디지털 시대를 가능하게 했다.:)'

In [33]:
runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    modified = lambda x: x['num'] + 1,
)

runnable.invoke({'num':1})

{'passed': {'num': 1}, 'modified': 2}

In [34]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified = add_smile,
)

runnable.invoke('안녕하세요')

{'passed': '안녕하세요', 'modified': '안녕하세요:)'}

In [35]:
model = ChatOpenAI(model='gpt-4o-mini', max_tokens=128, temperature=0)

history_prompt = ChatPromptTemplate.from_template("{topic}이 무엇의 약자인지 알려주세요")
celeb_prompt = ChatPromptTemplate.from_template("{topic}분야의 유명인사 3명의 이름만 알려주세요")

history_chain = history_prompt | model | StrOutputParser()
celeb_chain = celeb_prompt | model | StrOutputParser()

map_chain = RunnableParallel(history=history_chain, celeb=celeb_chain)

result = map_chain.invoke({"topic":"영화"})
result

{'history': '"영화"는 "영상"과 "화면"의 합성어로, 일반적으로 "영화"라는 용어는 특정한 약자가 아닙니다. 영화는 움직이는 이미지를 통해 이야기를 전달하는 예술 형식으로, 다양한 장르와 스타일이 존재합니다. "영화"라는 단어는 한국어에서 사용되는 일반적인 용어입니다.',
 'celeb': '1. 스티븐 스필버그 (Steven Spielberg)\n2. 마틴 스코세이지 (Martin Scorsese)\n3. 앤젤리나 졸리 (Angelina Jolie)'}